In [1]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 24 19:19:43 2019
@author: sb
"""
 
from keras.models import Sequential, Model
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.callbacks import TensorBoard
from keras.utils import plot_model
#from keras.utils.np_utils import to_categorical
#import matplotlib.pyplot as plt
 

Using TensorFlow backend.


In [2]:

num_classes = 2
img_rows,img_cols = 224,224
batch_size = 64
input_shape = (img_rows,img_cols,3)
 
datagen=ImageDataGenerator()   
train_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/train02',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)
 
#读验证集图片
validation_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/train02',
    classes=['bird','dog'],
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)
 
 

Found 2600 images belonging to 2 classes.
Found 2600 images belonging to 2 classes.


In [ ]:

vgg16 = VGG16(include_top=False, weights='imagenet',input_shape=input_shape)
    
for layer in vgg16.layers:
    layer.trainable = False
   # if layer.name=='block5_conv1':
   #     trainable=True
model = Sequential()
model.add(vgg16)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
#model.load_weights('outputs/weights_vgg16_use.h5')
 
model.summary()




14966784/58889256 [======>.......................] - ETA: 5:13:08

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer = optimizers.Adam(0.0001), metrics = ['accuracy'])


In [ ]:
#使用tensorboard查看训练過程
tb = TensorBoard(log_dir='/home/sb/kaggle/Dogs_vs_Cats',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
                 batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True, # 是否可视化梯度直方图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=0, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)
callbacks = [tb]
 
 
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=100,
        epochs=6,
        callbacks = callbacks,
        validation_data=validation_generator,
        validation_steps=32
       # callbacks=[TensorBoard(log_dir='logs/3')]
        )
 
model.save_weights('my_weights_vgg16_use.h5')